In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.optimizers import RMSprop
from keras.layers import Embedding, LSTM, Dense
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dropout
import pandas as pd
import tensorflow as tf
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
import pickle


# Модель на уже подготовленных данных. Тест=14 дней

In [ ]:
data_new_no_dupl = pd.read_csv('/kaggle/working/final_wo_dupl.csv',index_col=0)

In [ ]:
seasons_unique = data_new_no_dupl.season.unique()
season_dataframes = {}

for season in seasons_unique:
    season_dataframes[season] = data_new_no_dupl[data_new_no_dupl.season == season].copy()



In [ ]:

# функцию потерь wape_loss
def wape_loss(y_true, y_pred):
    y_true_float = K.cast(y_true, dtype='float32')
    error = K.abs(y_true_float - y_pred)
    denominator = K.abs(y_true_float) + K.abs(y_pred)
    return K.sum(error) / K.sum(denominator)
np.random.seed(42)
tf.random.set_seed(42)

lag_1_window = 7
lag_2_window = 9
seasons_unique = data_new.season.unique()
season_loss_dct = {}

In [ ]:

# Создаем новый DataFrame для хранения лагов|
for season in tqdm(season_dataframes.keys(), desc="Processing Season"):
    test_end_date = season_dataframes[season]['date'].max()
    # Вычисляем начальную дату, отсчитывая от конца
    test_start_date = test_end_date - timedelta(days=days_to_keep_in_test - 1)
    # Фильтруем данные для тестовой выборки
    data_new_test = season_dataframes[season][(season_dataframes[season]['date'] >= test_start_date) & (season_dataframes[season]['date'] <= test_end_date)]
    # Фильтруем данные для тренировочной выборки (оставляем все, кроме тестовой части)
    data_new_train = season_dataframes[season][season_dataframes[season]['date'] < test_start_date]
    lag_df_train = pd.DataFrame()
    # Перебираем уникальные пары магазин-товар
    for store in tqdm(data_new_train.st_id.unique(), desc="Processing Stores Train"):
        for product in data_new_train.pr_sku_id.unique():
            subset = data_new_train[(data_new_train['st_id'] == store) & (data_new_train['pr_sku_id'] == product)].copy()

            # Фильтруем DataFrame для текущей пары магазин-товар
            subset['lag_1_sales'] = subset['pr_sales_in_units'].shift(periods=1)
            subset['lag_2_sales'] = subset['pr_sales_in_units'].shift(periods=2)
            subset['lag_7_sales'] = subset['pr_sales_in_units'].shift(periods=7)
            subset['lag_14_sales'] = subset['pr_sales_in_units'].shift(periods=14)
            # Вычисляем скользящее среднее за 7 дней
#             subset['rolling_mean_1'] = subset['pr_sales_in_units'].shift().rolling(window=1, min_periods=1).mean()
            subset['rolling_mean_2'] = subset['pr_sales_in_units'].shift().rolling(window=2, min_periods=2).mean()
            subset['rolling_mean_7'] = subset['pr_sales_in_units'].shift().rolling(window=7, min_periods=7).mean()
            subset['rolling_mean_14'] = subset['pr_sales_in_units'].shift().rolling(window=14, min_periods=14).mean()

            # Заполняем пропущенные значения скользящим средним
#             subset['lag_1_sales'].fillna(rolling_mean_1, inplace=True)
#             subset['lag_2_sales'].fillna(rolling_mean_2, inplace=True)
#             subset['lag_7_sales'].fillna(rolling_mean_7, inplace=True)
#             subset['lag_14_sales'].fillna(rolling_mean_14, inplace=True)

            # Добавляем результаты в общий DataFrame
            lag_df_train = pd.concat([lag_df_train, subset])
            lag_df_train = lag_df_train.fillna(value=0)
    # Создаем новый DataFrame для хранения лагов
    lag_df_test = pd.DataFrame()

    # Перебираем уникальные пары магазин-товар
    for store in tqdm(data_new_test.st_id.unique(), desc="Processing Stores Test"):
        for product in data_new_test.pr_sku_id.unique():
            # Фильтруем DataFrame для текущей пары магазин-товар
            subset = data_new_test[(data_new_test['st_id'] == store) & (data_new_test['pr_sku_id'] == product)].copy()
            # Создаем лаги
            subset['lag_1_sales'] = subset['pr_sales_in_units'].shift(periods=1)
            subset['lag_2_sales'] = subset['pr_sales_in_units'].shift(periods=2)
            subset['lag_7_sales'] = subset['pr_sales_in_units'].shift(periods=7)
            subset['lag_14_sales'] = subset['pr_sales_in_units'].shift(periods=14)
            # Вычисляем скользящее среднее за 7 дней
#             subset['rolling_mean_1'] = subset['pr_sales_in_units'].shift().rolling(window=1, min_periods=1).mean()
            subset['rolling_mean_2'] = subset['pr_sales_in_units'].shift().rolling(window=2, min_periods=2).mean()
            subset['rolling_mean_7'] = subset['pr_sales_in_units'].shift().rolling(window=7, min_periods=7).mean()
            subset['rolling_mean_14'] = subset['pr_sales_in_units'].shift().rolling(window=14, min_periods=14).mean()
            # Заполняем пропущенные значения скользящим средним
            # Добавляем результаты в общий DataFrame
            lag_df_test = pd.concat([lag_df_test, subset])
            lag_df_test = lag_df_test.fillna(value=0)
    lag_df_train.drop(['date'],axis=1,inplace=True)
    lag_df_test.drop(['date'],axis=1,inplace=True)
    features_lag_train = lag_df_train.drop('pr_sales_in_units',axis=1)
    target_lag_train = lag_df_train.pr_sales_in_units
    features_lag_test = lag_df_test.drop('pr_sales_in_units',axis=1)
    target_lag_test = lag_df_test.pr_sales_in_units
    # Теперь у вас есть словарь season_dataframes, где ключами являются уникальные сезоны,
    # а значениями - соответствующие датафреймы.


    # Создаем объект LabelEncoder
    label_encoder = LabelEncoder()
    features_lag_train['st_id'] = label_encoder.fit_transform(features_lag_train['st_id'])
    features_lag_train['pr_sku_id'] = label_encoder.fit_transform(features_lag_train['pr_sku_id'])

    features_lag_test['st_id'] = label_encoder.fit_transform(features_lag_test['st_id'])
    features_lag_test['pr_sku_id'] = label_encoder.fit_transform(features_lag_test['pr_sku_id'])


    # Перевод признаков и целевых значений в тензоры
    features_train_tensor = tf.convert_to_tensor(features_lag_train.values, dtype=tf.float32)
    target_train_tensor = tf.convert_to_tensor(target_lag_train.values, dtype=tf.int64)
    # Перевод признаков и целевых значений в тензоры
    features_test_tensor = tf.convert_to_tensor(features_lag_test.values, dtype=tf.float32)
    target_test_tensor = tf.convert_to_tensor(target_lag_test.values, dtype=tf.int64)



    # Создаем модель LSTM
    model = Sequential()
    model.add(LSTM(64, input_shape=(features_train_tensor.shape[1], 1), return_sequences=True,kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(LSTM(32, activation='tanh'))
    model.add(Dropout(0.8))
    model.add(Dense(1, activation='relu'))


    # Компилируем модель с функцией потерь MSE
    model.compile(optimizer=RMSprop(learning_rate=0.001), loss='mean_absolute_error', metrics=[wape_loss])



    # Обучаем модель
    model.fit(np.expand_dims(features_train_tensor, axis=-1), target_train_tensor, epochs=1, batch_size=32, verbose=1)

    # Оцениваем модель на тестовой выборке
    test_loss = model.evaluate(np.expand_dims(features_test_tensor, axis=-1), target_test_tensor)
    season_loss_dct[season] = test_loss
    # pickle file Сохранение модели
    # model_filename = f"{season}.pkl"
    # with open(model_filename, 'wb') as model_file:
    #     pickle.dump(model, model_file)
    # print(f"Saved model to {model_filename}")
    print(f"For season_{season} data frame test Loss (WAPE): {test_loss}")

# График важности признаков

In [ ]:

results = []
print('Computing LSTM feature importance...')

#  базовые потери без перемешивания признаков
baseline_loss = model.evaluate(np.expand_dims(features_test_tensor, axis=-1), target_test_tensor, verbose=0)
results.append({'feature': 'BASELINE', 'loss': baseline_loss})

#  имена столбцов признаков
feature_names =  features_lag_test.columns

for feature_column in tqdm(range(features_test_tensor.shape[1])):
    #  значения признака для последующего восстановления
    save_col = tf.identity(features_test_tensor[:, feature_column])

    #  значения признака
    shuffled_col = tf.random.shuffle(features_test_tensor[:, feature_column])

    #  оригинальные значения перемешанными
    features_test_tensor = tf.concat([features_test_tensor[:, :feature_column], shuffled_col[:, tf.newaxis], features_test_tensor[:, feature_column + 1:]], axis=-1)

    #  потери модели после перемешивания признака
    test_loss = model.evaluate(np.expand_dims(features_test_tensor, axis=-1), target_test_tensor, verbose=0)

    # , что индекс в пределах допустимых значений
    if feature_column < len(feature_names):
        results.append({'feature': feature_names[feature_column], 'loss': test_loss})

    #  исходные значения признака
    features_test_tensor = tf.concat([features_test_tensor[:, :feature_column], save_col[:, tf.newaxis], features_test_tensor[:, feature_column + 1:]], axis=-1)




In [ ]:


print()
df = pd.DataFrame(results)
df = df.sort_values('loss')
plt.figure(figsize=(5, 10))
plt.barh(np.arange(len(df)), df.loss.apply(lambda x: float(x[0])))
plt.yticks(np.arange(len(df)), df.feature.values)
plt.title('LSTM Feature Importance', size=16)
plt.ylim((-1, len(df)))
plt.plot([baseline_loss, baseline_loss], [-1, len(df)], '--', color='orange',
         label=f'Baseline Loss\n{baseline_loss:.3f}')
plt.xlabel(f'Test Loss with feature permuted', size=14)
plt.ylabel('Feature', size=14)
plt.legend()
plt.show()
